In [1]:
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd 
import re

In [2]:
wordnet = WordNetLemmatizer()
stopwords = stopwords.words('english')

In [41]:
dataset = pd.read_csv('sample_data/testhatespeech.csv')
dataset.head()

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [42]:
def clean_text(text, stopword, max_length):
    text = text.lower()
    text = text.replace("'","")
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.split()
    text = [word for word in text if word not in stopword]
    text = [wordnet.lemmatize(word) for word in text]
    text = text[:max_length]
    return text

dataset["tweet"] = dataset["tweet"].apply(clean_text, stopword = stopwords, max_length = 100)

In [43]:
vocab = []
pos_vocab = []
neg_vocab = []
for i in range(len(dataset["tweet"])):
    if dataset["Toxicity"][i] == 0:
        neg_vocab += dataset["tweet"][i]
    else:
        pos_vocab += dataset["tweet"][i]
    vocab += dataset["tweet"][i]
vocab = set(vocab)
vocab = list(vocab)

In [44]:
df = pd.DataFrame(columns=['vocab', 'pos_prob', 'neg_prob'])
a = 1
for j in range(len(vocab)):
    df.loc[j, 'vocab'] = vocab[j]
    pos_prob = (pos_vocab.count(vocab[j]) + 1)/(len(pos_vocab) + len(vocab))
    neg_prob = (neg_vocab.count(vocab[j]) + 1)/(len(neg_vocab) + len(vocab))
    df.loc[j, "pos_prob"] = pos_prob
    df.loc[j, "neg_prob"] = neg_prob

df

,vocab,pos_prob,neg_prob
0,wonderful,0.000092,0.00041
1,thefuck,0.000092,0.000055
2,eradicate,0.000092,0.000055
3,blank,0.000092,0.000055
4,revival,0.000092,0.000055
...,...,...,...
8642,clever,0.000092,0.000082
8643,migrant,0.000092,0.000082
8644,weekendvibes,0.000092,0.000055
8645,dutycalls,0.000092,0.000055


In [45]:
df.to_csv('word_probability_test.csv', index=False)

In [ ]:
input = "love"
input = clean_text(input, stopwords, 1000)
prior_ratio_pos = sum(dataset["Toxicity"] == 1)/len(dataset)
prior_ratio_neg = sum(dataset["Toxicity"] == 0)/len(dataset)
pos_result = 0.5
neg_result = 0.5
print(prior_ratio_pos, prior_ratio_neg)
for index in range(len(input)):
    pos_result = pos_result*df["pos_prob"][df["vocab"].tolist().index(input[index])]
    neg_result = neg_result*df["neg_prob"][df["vocab"].tolist().index(input[index])]
    print(df["vocab"].tolist().index(input[index]))
print(pos_result > neg_result)